In [1]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import numpy as np

global num
num=str(6)
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = np.loadtxt('distance_'+str(num)+'.txt', dtype=np.float32, delimiter=',')
    data['distance_matrix']=np.dot(data['distance_matrix'],np.diag([100000000000000] * len(data['distance_matrix'])))
    demands=[]
    for i in np.loadtxt('demand_'+str(num)+'.txt', dtype=np.float32, delimiter=','):
        demands.append(int(i[1]))
    data['demands'] = demands
    w=np.loadtxt('spec_'+str(num)+'.txt', dtype=np.float32, delimiter=',')
    data['vehicle_capacities'] = []
    data['num_vehicles'] = int(w[1])
    for j in range(int(w[1])):
        data['vehicle_capacities'].append(int(w[2]))
    data['depot'] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    
    f=open("solution_"+num+".txt","w")
    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        l=[]
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            l.append(int('{0}'.format(node_index, route_load)))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        for i in range(len(l)):
            l[i]=l[i]+1
        l.append(1)
        kk=0
        for i in l:
            kk+=1
            if kk==len(l):
                f.write(str(i))
            else:
                f.write(str(i)+',')
        f.write('\n')
        print(l)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))
    f.close()


def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_MOST_CONSTRAINED_ARC)

    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.seconds = 90000
    search_parameters.log_search = True
    search_parameters.use_full_propagation=True
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == '__main__':
    main()

Objective: 7701300869202613623
[1, 89, 414, 787, 623, 150, 457, 161, 929, 311, 940, 575, 822, 264, 527, 260, 905, 890, 832, 758, 499, 304, 230, 536, 143, 375, 818, 1]
Route for vehicle 0:
 0 Load(0) ->  88 Load(2) ->  413 Load(9) ->  786 Load(18) ->  622 Load(25) ->  149 Load(30) ->  456 Load(32) ->  160 Load(39) ->  928 Load(46) ->  310 Load(56) ->  939 Load(58) ->  574 Load(63) ->  821 Load(69) ->  263 Load(71) ->  526 Load(73) ->  259 Load(75) ->  904 Load(77) ->  889 Load(79) ->  831 Load(82) ->  757 Load(91) ->  498 Load(99) ->  303 Load(102) ->  229 Load(110) ->  535 Load(114) ->  142 Load(122) ->  374 Load(128) ->  817 Load(131) ->  0 Load(131)
Distance of the route: 174833353614807119m
Load of the route: 131

[1, 187, 682, 66, 153, 371, 390, 531, 327, 619, 324, 952, 138, 128, 997, 522, 47, 127, 788, 730, 692, 349, 830, 513, 481, 247, 393, 899, 562, 1]
Route for vehicle 1:
 0 Load(0) ->  186 Load(3) ->  681 Load(5) ->  65 Load(14) ->  152 Load(18) ->  370 Load(25) ->  389 Load(3